In [39]:
import sys
sys.path.append("../")
from src.tasks import tasks 
from src.models import RNN_Stack
import numpy as np
import torch

In [18]:
v = torch.tensor(np.asarray([1,0,0,1]))

In [19]:
tasks.get_parity(v, 2)

tensor(1)

In [33]:
seq, label = tasks.make_batch_Nbit_pair_parity([2], 2)
print(seq.shape, label)
# one readout head with 

torch.Size([2, 4, 1]) [tensor([0, 0])]


In [36]:
seq, label = tasks.make_batch_Nbit_pair_parity([2,1, 3], 2)
print(seq.shape, label)

tensor([[[1.],
         [1.],
         [1.],
         [0.],
         [1.],
         [1.]],

        [[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]]]) torch.Size([2, 6, 1]) [tensor([0, 0]), tensor([1, 0]), tensor([0, 0])]


In [37]:
seq, label = tasks.make_batch_Nbit_pair_parity([2,1, 3], 2)
print(seq.shape, label)

torch.Size([2, 7, 1]) [tensor([0, 0]), tensor([0, 1]), tensor([1, 0])]


In [53]:
rnn = RNN_Stack(input_size=1, net_size=[5], num_classes=2, bias=True, num_readout_heads=4, tau=1., train_tau=True).cuda()
# need to revisit tau later. 

In [54]:
rnn

RNN_Stack(
  (parameterlist): ParameterList(  (0): Parameter containing: [torch.float32 of size 5 (cuda:0)])
  (modulelist): ModuleList(
    (0): Linear(in_features=1, out_features=5, bias=True)
    (1): Linear(in_features=5, out_features=5, bias=True)
    (2-5): 4 x Linear(in_features=5, out_features=2, bias=True)
  )
  (afunc): LeakyReLU(negative_slope=0.01)
)

In [63]:
last_time, heads = rnn.forward(seq.cuda())

In [65]:
last_time

[tensor([[ 8.7356e-01,  1.1081e+00,  8.5656e-01,  1.1935e-01,  1.2735e+00],
         [ 2.1628e-01,  3.1776e-01, -5.8489e-05, -2.3078e-03,  8.2988e-01],
         [ 6.6754e-01,  8.6797e-01,  6.4337e-01,  1.3202e-01,  1.0925e+00],
         [ 6.5417e-01,  8.8313e-01,  6.4919e-01,  1.3308e-01,  1.0809e+00],
         [ 4.1405e-01,  5.7505e-01,  2.0639e-01, -2.2553e-03,  1.0220e+00],
         [ 6.7186e-01,  8.6141e-01,  6.4485e-01,  1.2554e-01,  1.0841e+00],
         [ 6.6769e-01,  8.6478e-01,  6.4455e-01,  1.2752e-01,  1.0838e+00]],
        device='cuda:0', grad_fn=<LeakyReluBackward0>)]

In [66]:
heads

[tensor([[ 0.2284, -0.5698],
         [-0.2273, -0.1739],
         [ 0.1060, -0.4666],
         [ 0.1138, -0.4656],
         [-0.1084, -0.2697],
         [ 0.1097, -0.4702],
         [ 0.1095, -0.4686]], device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([[-1.1219, -0.1929],
         [-0.5631, -0.1147],
         [-0.9558, -0.1912],
         [-0.9496, -0.1904],
         [-0.7362, -0.1119],
         [-0.9518, -0.1937],
         [-0.9512, -0.1929]], device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([[ 0.2901, -0.3687],
         [ 0.4958, -0.1704],
         [ 0.3488, -0.3065],
         [ 0.3535, -0.2928],
         [ 0.4513, -0.2235],
         [ 0.3418, -0.3091],
         [ 0.3447, -0.3059]], device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([[ 0.1930, -0.4410],
         [-0.0158, -0.0694],
         [ 0.1280, -0.3616],
         [ 0.1304, -0.3678],
         [ 0.0449, -0.1547],
         [ 0.1313, -0.3585],
         [ 0.1305, -0.3600]], device='cuda:0', grad_fn=<AddmmBackward0>)]

In [70]:
print(len(heads), heads[0].shape)

4 torch.Size([7, 2])


In [68]:
seq.shape

torch.Size([2, 7, 1])

In [72]:
# classify_in_time useful for MI calcs
